In [82]:
DEV=True
def import_module_py(name):
    if name not in bpy.data.texts:
        m_t = bpy.data.texts.new(name=name)
        print("name: {name}")
        with open(name) as f:
            s = ''.join(f.readlines())
            print(s)
            m_t.write(s)

In [83]:
try: 
    bpy.data.texts.remove(bpy.data.texts["qsl.py"])
except: pass
import_module_py("qsl.py")
from types import ModuleType

name: {name}
from typing import IO, Optional
import re
import os

Q_CONT_SOLID = 1                # /* an eye is never valid in a solid */
Q_CONT_LAVA = 0x8
Q_CONT_SLIME = 0x10
Q_CONT_WATER = 0x20
Q_CONT_FOG = 0x40
Q_CONT_AREAPORTAL = 0x8000
Q_CONT_PLAYERCLIP = 0x10000
Q_CONT_MONSTERCLIP = 0x20000
Q_CONT_TELEPORTER = 0x40000
Q_CONT_JUMPPAD = 0x80000
Q_CONT_CLUSTERPORTAL = 0x100000
Q_CONT_DONOTENTER = 0x200000
Q_CONT_BOTCLIP = 0x400000
Q_CONT_ORIGIN = 0x1000000       # /* removed before bsping an entity */
Q_CONT_BODY = 0x2000000         # /* should never be on a brush, only in game */
Q_CONT_CORPSE = 0x4000000
Q_CONT_DETAIL = 0x8000000       # /* brushes not used for the bsp */
Q_CONT_STRUCTURAL = 0x10000000  # /* brushes used for the bsp */
Q_CONT_TRANSLUCENT = 0x20000000 # /* don't consume surface fragments inside */
Q_CONT_TRIGGER = 0x40000000
Q_CONT_NODROP = 0x80000000      # /* don't leave bodies or items (death fog, lava) */
Q_SURF_NODAMAGE = 0x1           # /* never give falling

In [1]:
import bpy
import zipfile
import json
try:
    import qsl
except ImportError:
    qsl = bpy.data.texts["qsl.py"].as_module()

def add_target_arrow():
    import gpu
    import mathutils as m
    from gpu_extras.batch import batch_for_shader
    vertex_shader = '''
        uniform mat4 viewProjectionMatrix;
        uniform vec3 beg;
        uniform vec3 end;
        uniform vec3 cam;

        in vec3 position;
        in vec4 color;

        void main()
        {
            vec3 pos = mix(beg, end, position);
            vec3 l = normalize(end - beg);
            vec3 c = (beg + end) * 0.5f;
            vec3 o = normalize(cross(c - cam, l));
            
            vec3 arr = c - l * 0.5f + color.y * o;
            
            gl_Position = viewProjectionMatrix * vec4(mix(arr, pos, color.x), 1.0f);
        }
    '''
    fragment_shader = '''
        uniform vec4 color;

        //in vec4 vc;
        out vec4 FragColor;

        void main()
        {
            FragColor = color;
        }
    '''
    handle_name = "__arrow_draw_handle"

    coords = [(0,0,0), (1,1,1), (0.5,0.5,0.5), (0, 0, 0), (0.5, 0.5, 0.5), (0, 0, 0)]
    colors = [(1,0,0,0), (1,0,0,0), (1,0,0,0), (0,0.5,0,20),(1,0,0,0), (0,-0.5,0,20)]
    shader = gpu.types.GPUShader(vertex_shader, fragment_shader)
    batch = batch_for_shader(shader, 'LINES', {"position": coords, "color": colors})
    def draw():
        shader.bind()
        matrix = bpy.context.region_data.perspective_matrix
        cam = m.Matrix.inverted(bpy.context.region_data.view_matrix) @ m.Vector((0,0,0,1))
        shader.uniform_float("viewProjectionMatrix", matrix)
        shader.uniform_float("cam", cam.xyz)
        
        drawn = set()
        for ob in bpy.context.selected_objects:
            if ob.target and ob.target_warning == '':
                tloc = ob.target.location
                shader.uniform_float("beg", ob.location)
                shader.uniform_float("end", tloc)
                shader.uniform_float("color", (0.2, 0.95, 0.1, 1))
                batch.draw(shader)
                drawn.add((ob.name, ob.target.name))
                
        for ob in bpy.context.selected_objects:
            for i in bpy.data.objects:
                if i.target != ob: continue
                if (i.name, ob.name) in drawn: continue # already drawn
                tloc = i.location
                shader.uniform_float("beg", tloc)
                shader.uniform_float("end", ob.location)
                shader.uniform_float("color", (0.7, 0.55, 0.1, 1))
                batch.draw(shader)
    try:
        if handle_name in bpy.app.driver_namespace:
            bpy.types.SpaceView3D.draw_handler_remove(bpy.app.driver_namespace[handle_name], "WINDOW")
    except:
        pass
    handle = bpy.types.SpaceView3D.draw_handler_add(draw, (), 'WINDOW', 'POST_VIEW')
    bpy.app.driver_namespace[handle_name] = handle

def to_uint(i):
    return -(i & 0x7fffffff)-1 if (i & 0x80000000) != 0 else i
    
def setup_groups(shaders, textures, dst_collection):
    groups = set(shaders.keys())
    groups = groups.union(set(textures.keys()))

    def_contents, def_flags = qsl.build_surfparam([])

    for group in groups:
        tg = dst_collection.add()
        tg.name = group
        # write shaders
        for sh in shaders.get(group, []):
            tx = tg.textures.add()
            tx.name = os.path.basename(sh["name"]) 
            tx.is_shader = True
            tx.contents = to_uint(sh["contents"])
            tx.flags = to_uint(sh["flags"])
            tx.pak = sh["pak"]
            tx.path = sh["name"]
            tx.shader = json.dumps(sh)

        for t in textures.get(group, []):
            tx = tg.textures.add()
            tx.name = os.path.basename(t["path"]) 
            tx.is_shader = False
            tx.contents = def_contents
            tx.flags = def_flags
            tx.pak = t["pak"]
            tx.path = t["path"]
            tx.shader = ""

def create_material(tex):
    nm = bpy.data.materials.new(name=qsl.to_name(tex.path))
    nm.use_nodes = True
    img_n = nm.node_tree.nodes.new(type="ShaderNodeTexImage")
    img_n.image = tex.image
    nm.node_tree.links.new(nm.node_tree.nodes['Principled BSDF'].inputs['Base Color'], img_n.outputs['Color'])
    nm.texture = tex

    return nm

def load_pk3_image(pak: zipfile.ZipFile, path, tmp_path):
    filename = pak.extract(path, tmp_path)
    
    fn = qsl.to_name(path)
    limg = bpy.data.images.load(filename, check_existing=True)
    limg.name = fn
    # limg.pack()
    return limg

def on_executable_changed(scene, context):
    import os
    # look for paks and parse shaders & texture groups
    pth = scene.executable_path
    if not os.path.isfile(pth): return # [TODO] add information message (paks parsed, etc)
    dpath = os.path.dirname(pth)
    allsh, alltx = {}, {}
    for i in os.listdir(dpath):
        d = os.path.join(dpath, i)
        if not os.path.isdir(d): continue
        print(f"Mod found: '{i}'")
        for j in os.listdir(d):
            f = os.path.join(d, j)
            if not os.path.isfile(f) or not f.endswith('.pk3'): continue
            print(f"Pak found: '{j}'")
            with zipfile.ZipFile("/home/rantrave/games/Quake3/baseq3/pak0.pk3", 'r') as pak:
                sh, tx = qsl.list_textures(pak, dpath)
            qsl.append_groups(allsh, sh)
            qsl.append_groups(alltx, tx)

    setup_groups(allsh, alltx, scene.texture_collection)

def on_collection_changed(scene, context):
    # load images from paks
    tx_group = 

def on_preview_changed(scene, context):
    pass

def on_target_update(object, context):
#    if object.type != 'MESH':
#        object.target_warning = "Trigger should be a mesh"
#        return

    if object.target == object: # self reference
        object.target_warning = "Object points itself"
        return
    
    object.target_warning = ""

# displayed as: [miniature] Name [pak:path]
# at preview: [full image on texture], pak, path, flags, content, shader
class BSP_Texture(bpy.types.PropertyGroup):
    # short name to display
    name: bpy.props.StringProperty()
    is_shader: bpy.props.BoolProperty()
    image: bpy.props.PointerProperty(type=bpy.types.Image)
    # flags
    contents: bpy.props.IntProperty()
    flags: bpy.props.IntProperty()
    # refenence data
    pak: bpy.props.StringProperty()
    path: bpy.props.StringProperty()
    # json of the shader, empty if is a texture
    shader: bpy.props.StringProperty()

class BSP_TextureGroup(bpy.types.PropertyGroup):
    name: bpy.props.StringProperty()
    textures: bpy.props.CollectionProperty(type=BSP_Texture)


def map_resources(scene):
    for i in scene.texture_collection:
        for j in i.textures:
            yield (j.path, j.pak)


class BSP_PT_MaterialPanel(bpy.types.Panel):
    bl_space_type = 'PROPERTIES'
    bl_region_type = 'WINDOW'
    bl_context = "object"
    bl_label = "BSP Object"
    def draw(self, context):
        ob = context.object
        layout = self.layout
        col = layout.column()
        row = col.row(align=True)
        row.label(text="Object type")
        row.prop_menu_enum(ob, "object_type", text=ob.object_type)
        if ob.object_type == 'VOLUME':
            col.separator()
            row = col.row(align=True)
            row.prop(ob, "volume_type", text=ob.volume_type)
            
            if ob.volume_type == 'FOG':
                row = col.row(align=True)
                row.label(text="Fog color")
                row.prop(ob, "fog_color", text="")
                row = col.row(align=True)
                row.label(text="Fog distance")
                row.prop(ob, "fog_distance", text="")
        if ob.object_type == 'TRIGGER':
            row = col.row(align=True)
            row.label(text="Target")
            row.prop(ob, "target", text="")
            if ob.target_warning != '':
                row = col.row(align=True)
                row.label(text=ob.target_warning, icon='ERROR')
            
    
classes = [
    BSP_Texture,
    BSP_TextureGroup,
    BSP_PT_MaterialPanel,
]

props = [
    (bpy.types.Scene, {
        "texture_collection": bpy.props.CollectionProperty(type=BSP_TextureGroup),
        "texture_collection_current": bpy.props.IntProperty(update=on_collection_changed),
        "texture_preview_current": bpy.props.IntProperty(),
        "texture_preview": bpy.props.PointerProperty(type=bpy.types.Texture, update=on_preview_changed),
        "executable_path": bpy.props.StringProperty(), # update=on_executable_changed), # not allow to edit as text
    }),
    (bpy.types.Material, {
        "material_type": bpy.props.EnumProperty(
            description="Special type of surface",
            items = [
                ('DETAIL', 'Detail', "Detail face without collision"),
                ('STRUCTURAL', 'Structural', "Structural face, that affects bsp tree and visibility"),
                ('PORTAL', 'Portal', "Face that only affects bsp, may be used to separate zones"),
            ]
        ),
        # default face properties
        "texture":      bpy.props.PointerProperty(type=bpy.types.Texture),
        "translucent":  bpy.props.BoolProperty(),
        "slick":        bpy.props.BoolProperty(),
        "noimpact":     bpy.props.BoolProperty(),
        "nomarks":      bpy.props.BoolProperty(),
        "nodamage":     bpy.props.BoolProperty(),
        "dust":         bpy.props.BoolProperty(),
        "surface_type": bpy.props.EnumProperty(
            description = "Steps sound",
            items = [('DEFAULT', 'Default', "Default sound"),
                    ('METAL', 'Metal', "Metal steps"),
                    ('FLESH', 'Flesh', "Flesh sound"),
                    ('NOSOUND', 'NoSound', "No sound")]
        ),
        # portal properties
        "zone_portal":  bpy.props.BoolProperty(),
    }),
    (bpy.types.Object, {
        "object_type":  bpy.props.EnumProperty(
            description="Object properties",
            items=[
                ('GEOMETRY', 'Geometry', "Default geometry of the level"),
                ('ENTITY', 'Entity', "Entity (door, lift, etc)"),
                ('VOLUME', 'Volume', "Special type of block"),
                ('TRIGGER', 'Trigger', "Triggers an event"),
                ('ZONEINFO', 'ZoneInfo', "Marks areas")
            ]
        ),
        # [TODO] entity params
        # volume params
        "volume_type":    bpy.props.EnumProperty(
            items=[
                ('PLAYERCLIP', 'PlayerClip', "Blocks player, pass shots"),
                ('FULLCLIP', 'FullClip', "Blocks everything"),
                ('WATER', 'Water', "Swimmable water"),
                ('LAVA', 'Lava', "Lava"),
                ('SLIME', 'Slime', "Slime"),
                ('FOG', 'Fog', "Fog"),
                ('NODROP', 'NoDrop', "No drops are inside"),
                # [TODO] more
            ]
        ),
        "fog_color":    bpy.props.FloatVectorProperty(subtype='COLOR', size=3, default=[0.3,0.3,0.3]),
        "fog_distance": bpy.props.IntProperty(min=0, max=10000),

        # trigger params
        "target": bpy.props.PointerProperty(type=bpy.types.Object, update=on_target_update),
        "target_warning": bpy.props.StringProperty(),
        # zoneinfo params
    })
]



def register():
    for cls in classes:
        bpy.utils.register_class(cls)
    
    for tp, d in props:
        for k, v in d.items():
            setattr(tp, k, v)
    
    add_target_arrow()

def unregister():
    for cls in classes:
        clst = cls.__bases__[0].bl_rna_get_subclass_py(cls.__name__)
        if clst:
            bpy.utils.unregister_class(clst)

    for tp, d in props:
        for k in d:
            if hasattr(tp, k):
                delattr(tp, k)

if __name__ == '__main__':
    unregister()
    register()

SyntaxError: invalid syntax (3781157021.py, line 161)

In [40]:
unregister()
register()

In [42]:
cube = bpy.context.scene.objects["Cube"]

bpy.data.objects['Cube'].material_slots[0]

In [11]:
a = {"qwe": [1], "asd": [22]}
b = {"asd": [2], "zxc": [3]}
set(a).union(set(b))

{'asd', 'qwe', 'zxc'}

In [81]:
unregister()
register()
bpy.data.scenes[0].texture_collection.clear()

In [99]:
[x for x in pk3.filelist if x.filename.startswith("scripts")]

[<ZipInfo filename='scripts/' external_attr=0x10>,
 <ZipInfo filename='scripts/arenas.txt' compress_type=deflate external_attr=0x20 file_size=3321 compress_size=778>,
 <ZipInfo filename='scripts/base.shader' compress_type=deflate external_attr=0x20 file_size=2247 compress_size=445>,
 <ZipInfo filename='scripts/base_button.shader' compress_type=deflate external_attr=0x20 file_size=777 compress_size=246>,
 <ZipInfo filename='scripts/base_floor.shader' compress_type=deflate external_attr=0x20 file_size=26459 compress_size=2682>,
 <ZipInfo filename='scripts/base_light.shader' compress_type=deflate external_attr=0x20 file_size=44288 compress_size=2659>,
 <ZipInfo filename='scripts/base_object.shader' compress_type=deflate external_attr=0x20 file_size=2555 compress_size=446>,
 <ZipInfo filename='scripts/base_support.shader' compress_type=deflate external_attr=0x20 file_size=4914 compress_size=796>,
 <ZipInfo filename='scripts/base_trim.shader' compress_type=deflate external_attr=0x20 file_si

In [3]:
import qsl
import zipfile
import json
pk3 = zipfile.ZipFile("/home/rantrave/games/Quake3/defrag/hunterun-28.pk3", 'r')
for sf in [x for x in pk3.filelist if x.filename.endswith('.shader') and x.filename.startswith('scripts')]:
    print(sf)
    for sh in qsl.parse_shader(pk3.read(sf).decode('utf-8')):
        print(qsl.get_image(sh))
        # print(sh)

<ZipInfo filename='scripts/cellar2.shader' compress_type=deflate file_size=9057 compress_size=889>
textures/cellar2/weapspawn01.tga
textures/cellar2/tmpjp_green.tga
textures/cellar2/tmpjp_red.tga
textures/cellar2/tmpjp_violet.tga
textures/cellar2/tmpjp_yellow.tga
textures/cellar2/cretelighttrim01.tga
textures/cellar2/cretelighttrim01_green.tga
textures/cellar2/cretelighttrim01_red.tga
textures/cellar2/cretelighttrim01_violet.tga
textures/cellar2/cretelighttrim01_yellow.tga
textures/cellar2/cretewalllight01bis.tga
textures/cellar2/cretewalllight01bis_green.tga
textures/cellar2/cretewalllight01bis_red.tga
textures/cellar2/cretewalllight01bis_violet.tga
textures/cellar2/cretewalllight01bis_yellow.tga
textures/cellar2/glass_blue.tga
textures/cellar2/glass_green.tga
textures/cellar2/glass_red.tga
textures/cellar2/glass_yellow.tga
textures/cellar2/glass_violet.tga
textures/cellar2/circle.tga
textures/cellar2/halzap2a.tga
<ZipInfo filename='scripts/etpack_df_handbreaker4.shader' compress_type

In [87]:
pk3.extract("textures/base_button/shootme_glow.jpg", "./tmp.jpg")

'tmp.jpg/textures/base_button/shootme_glow.jpg'

In [17]:
def suggest_name(names, srcname):
    tx = [x for x in names if x == srcname]
    if len(tx) > 0:
        used = set()
        for i in [x.split('.')[-1] for x in names if x.startswith(srcname + '.')]:
            try:
                v = int(i)
                used.add(v)
            except ValueError:
                continue
        for i in range(1, 2 + len(used)):
            if i in used: continue
            return f'{srcname}.{i:003}'
        return None # impossible
    return srcname

In [18]:
suggest_name(['asd', 'asd.001'], 'asd')

'asd.002'

In [83]:
enumerate_textures(pk3, '', bpy.data.scenes[0].texture_collection)

1 0
1 0
0 16384
1 0
1 0
1 0
1 0
1 0
1 0
0 16384
0 16384
0 16384
0 16384
0 16384
0 16384
0 16416
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 32
1 0
1 0
1 4096
1 0
1 3088
1 0
1 0
1 0
1 0
536870976 19456
8 19472
8 3088
8 3088
1 0
1 0
1 0
1 3072
1 3072
0 19456
1 1
1 4096
1 0
1 0
1 0
1 1
1 0
1 1
1 1
1 0
1 0
1 0
1 0
1 0
2684354624 19456
1 0
1 0
0 19456
1 0
1 0
1 0
1 3072
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 16384
1 0
1 0
1 0
1 0
1 0
1 0
536870913 65536
536870913 65536
1 65536
1 0
1 0
0 16384
1 0
0 16384
1 0
1 0
1 0
1 0
1 0
1 0
1 0
536870913 0
1 0
1 0
1 0
1 0
1 0
536870913 3072
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
536870920 3088
536870920 16
536870944 16384
8 3088
536870944 19456
536870944 16384
536870944 16384
536870944 16384
536870944 16384
536870944 16384
1 0
1 0
536870920 3088
1 134160
536870920 3088
536870920 3088
536870920 3088
8 3088
536870920 3088
536870920 3088
536870920 3088
8 3088
536870944 19456
536870944 19456
536870944 19456
536870944 16384
536870944 19456
5368

In [96]:
with open('temp.py', 'w') as f:
    f.write(bpy.data.texts["panel_script.py"].as_string())

In [92]:
with open('temp2.py', 'w') as f:
    f.write(bpy.data.texts["testline.py"].as_string())

In [5]:
pk3.filename

'/home/rantrave/games/Quake3/baseq3/pak0.pk3'

In [7]:
import os


'baseq3/pak0.pk3'

In [93]:
for i in range(0,3):
    print(i)
    
else:
    print("else")

0
1
2
else


In [94]:
l = ('NONE', 'QUESTION', 'ERROR', 'CANCEL', 'TRIA_RIGHT', 'TRIA_DOWN', 'TRIA_LEFT', 'TRIA_UP', 'ARROW_LEFTRIGHT', 'PLUS', 'DISCLOSURE_TRI_RIGHT', 'DISCLOSURE_TRI_DOWN', 'RADIOBUT_OFF', 'RADIOBUT_ON', 'MENU_PANEL', 'BLENDER', 'GRIP', 'DOT', 'COLLAPSEMENU', 'X', 'DUPLICATE', 'TRASH', 'COLLECTION_NEW', 'OPTIONS', 'NODE', 'NODE_SEL', 'WINDOW', 'WORKSPACE', 'RIGHTARROW_THIN', 'BORDERMOVE', 'VIEWZOOM', 'ADD', 'REMOVE', 'PANEL_CLOSE', 'COPY_ID', 'EYEDROPPER', 'CHECKMARK', 'AUTO', 'CHECKBOX_DEHLT', 'CHECKBOX_HLT', 'UNLOCKED', 'LOCKED', 'UNPINNED', 'PINNED', 'SCREEN_BACK', 'RIGHTARROW', 'DOWNARROW_HLT', 'FCURVE_SNAPSHOT', 'OBJECT_HIDDEN', 'TOPBAR', 'STATUSBAR', 'PLUGIN', 'HELP', 'GHOST_ENABLED', 'COLOR', 'UNLINKED', 'LINKED', 'HAND', 'ZOOM_ALL', 'ZOOM_SELECTED', 'ZOOM_PREVIOUS', 'ZOOM_IN', 'ZOOM_OUT', 'DRIVER_DISTANCE', 'DRIVER_ROTATIONAL_DIFFERENCE', 'DRIVER_TRANSFORM', 'FREEZE', 'STYLUS_PRESSURE', 'GHOST_DISABLED', 'FILE_NEW', 'FILE_TICK', 'QUIT', 'URL', 'RECOVER_LAST', 'THREE_DOTS', 'FULLSCREEN_ENTER', 'FULLSCREEN_EXIT', 'BRUSHES_ALL', 'LIGHT', 'MATERIAL', 'TEXTURE', 'ANIM', 'WORLD', 'SCENE', 'OUTPUT', 'SCRIPT', 'PARTICLES', 'PHYSICS', 'SPEAKER', 'TOOL_SETTINGS', 'SHADERFX', 'MODIFIER', 'BLANK1', 'FAKE_USER_OFF', 'FAKE_USER_ON', 'VIEW3D', 'GRAPH', 'OUTLINER', 'PROPERTIES', 'FILEBROWSER', 'IMAGE', 'INFO', 'SEQUENCE', 'TEXT', 'SPREADSHEET', 'SOUND', 'ACTION', 'NLA', 'PREFERENCES', 'TIME', 'NODETREE', 'CONSOLE', 'TRACKER', 'ASSET_MANAGER', 'NODE_COMPOSITING', 'NODE_TEXTURE', 'NODE_MATERIAL', 'UV', 'OBJECT_DATAMODE', 'EDITMODE_HLT', 'UV_DATA', 'VPAINT_HLT', 'TPAINT_HLT', 'WPAINT_HLT', 'SCULPTMODE_HLT', 'POSE_HLT', 'PARTICLEMODE', 'TRACKING', 'TRACKING_BACKWARDS', 'TRACKING_FORWARDS', 'TRACKING_BACKWARDS_SINGLE', 'TRACKING_FORWARDS_SINGLE', 'TRACKING_CLEAR_BACKWARDS', 'TRACKING_CLEAR_FORWARDS', 'TRACKING_REFINE_BACKWARDS', 'TRACKING_REFINE_FORWARDS', 'SCENE_DATA', 'RENDERLAYERS', 'WORLD_DATA', 'OBJECT_DATA', 'MESH_DATA', 'CURVE_DATA', 'META_DATA', 'LATTICE_DATA', 'LIGHT_DATA', 'MATERIAL_DATA', 'TEXTURE_DATA', 'ANIM_DATA', 'CAMERA_DATA', 'PARTICLE_DATA', 'LIBRARY_DATA_DIRECT', 'GROUP', 'ARMATURE_DATA', 'COMMUNITY', 'BONE_DATA', 'CONSTRAINT', 'SHAPEKEY_DATA', 'CONSTRAINT_BONE', 'CAMERA_STEREO', 'PACKAGE', 'UGLYPACKAGE', 'EXPERIMENTAL', 'BRUSH_DATA', 'IMAGE_DATA', 'FILE', 'FCURVE', 'FONT_DATA', 'RENDER_RESULT', 'SURFACE_DATA', 'EMPTY_DATA', 'PRESET', 'RENDER_ANIMATION', 'RENDER_STILL', 'LIBRARY_DATA_BROKEN', 'BOIDS', 'STRANDS', 'LIBRARY_DATA_INDIRECT', 'GREASEPENCIL', 'LINE_DATA', 'LIBRARY_DATA_OVERRIDE', 'GROUP_BONE', 'GROUP_VERTEX', 'GROUP_VCOL', 'GROUP_UVS', 'FACE_MAPS', 'RNA', 'RNA_ADD', 'MOUSE_LMB', 'MOUSE_MMB', 'MOUSE_RMB', 'MOUSE_MOVE', 'MOUSE_LMB_DRAG', 'MOUSE_MMB_DRAG', 'MOUSE_RMB_DRAG', 'MEMORY', 'PRESET_NEW', 'DECORATE', 'DECORATE_KEYFRAME', 'DECORATE_ANIMATE', 'DECORATE_DRIVER', 'DECORATE_LINKED', 'DECORATE_LIBRARY_OVERRIDE', 'DECORATE_UNLOCKED', 'DECORATE_LOCKED', 'DECORATE_OVERRIDE', 'FUND', 'TRACKER_DATA', 'HEART', 'ORPHAN_DATA', 'USER', 'SYSTEM', 'SETTINGS', 'OUTLINER_OB_EMPTY', 'OUTLINER_OB_MESH', 'OUTLINER_OB_CURVE', 'OUTLINER_OB_LATTICE', 'OUTLINER_OB_META', 'OUTLINER_OB_LIGHT', 'OUTLINER_OB_CAMERA', 'OUTLINER_OB_ARMATURE', 'OUTLINER_OB_FONT', 'OUTLINER_OB_SURFACE', 'OUTLINER_OB_SPEAKER', 'OUTLINER_OB_FORCE_FIELD', 'OUTLINER_OB_GROUP_INSTANCE', 'OUTLINER_OB_GREASEPENCIL', 'OUTLINER_OB_LIGHTPROBE', 'OUTLINER_OB_IMAGE', 'OUTLINER_COLLECTION', 'RESTRICT_COLOR_OFF', 'RESTRICT_COLOR_ON', 'HIDE_ON', 'HIDE_OFF', 'RESTRICT_SELECT_ON', 'RESTRICT_SELECT_OFF', 'RESTRICT_RENDER_ON', 'RESTRICT_RENDER_OFF', 'RESTRICT_INSTANCED_OFF', 'OUTLINER_DATA_EMPTY', 'OUTLINER_DATA_MESH', 'OUTLINER_DATA_CURVE', 'OUTLINER_DATA_LATTICE', 'OUTLINER_DATA_META', 'OUTLINER_DATA_LIGHT', 'OUTLINER_DATA_CAMERA', 'OUTLINER_DATA_ARMATURE', 'OUTLINER_DATA_FONT', 'OUTLINER_DATA_SURFACE', 'OUTLINER_DATA_SPEAKER', 'OUTLINER_DATA_LIGHTPROBE', 'OUTLINER_DATA_GP_LAYER', 'OUTLINER_DATA_GREASEPENCIL', 'GP_SELECT_POINTS', 'GP_SELECT_STROKES', 'GP_MULTIFRAME_EDITING', 'GP_ONLY_SELECTED', 'GP_SELECT_BETWEEN_STROKES', 'MODIFIER_OFF', 'MODIFIER_ON', 'ONIONSKIN_OFF', 'ONIONSKIN_ON', 'RESTRICT_VIEW_ON', 'RESTRICT_VIEW_OFF', 'RESTRICT_INSTANCED_ON', 'MESH_PLANE', 'MESH_CUBE', 'MESH_CIRCLE', 'MESH_UVSPHERE', 'MESH_ICOSPHERE', 'MESH_GRID', 'MESH_MONKEY', 'MESH_CYLINDER', 'MESH_TORUS', 'MESH_CONE', 'MESH_CAPSULE', 'EMPTY_SINGLE_ARROW', 'LIGHT_POINT', 'LIGHT_SUN', 'LIGHT_SPOT', 'LIGHT_HEMI', 'LIGHT_AREA', 'CUBE', 'SPHERE', 'CONE', 'META_PLANE', 'META_CUBE', 'META_BALL', 'META_ELLIPSOID', 'META_CAPSULE', 'SURFACE_NCURVE', 'SURFACE_NCIRCLE', 'SURFACE_NSURFACE', 'SURFACE_NCYLINDER', 'SURFACE_NSPHERE', 'SURFACE_NTORUS', 'EMPTY_AXIS', 'STROKE', 'EMPTY_ARROWS', 'CURVE_BEZCURVE', 'CURVE_BEZCIRCLE', 'CURVE_NCURVE', 'CURVE_NCIRCLE', 'CURVE_PATH', 'LIGHTPROBE_CUBEMAP', 'LIGHTPROBE_PLANAR', 'LIGHTPROBE_GRID', 'COLOR_RED', 'COLOR_GREEN', 'COLOR_BLUE', 'TRIA_RIGHT_BAR', 'TRIA_DOWN_BAR', 'TRIA_LEFT_BAR', 'TRIA_UP_BAR', 'FORCE_FORCE', 'FORCE_WIND', 'FORCE_VORTEX', 'FORCE_MAGNETIC', 'FORCE_HARMONIC', 'FORCE_CHARGE', 'FORCE_LENNARDJONES', 'FORCE_TEXTURE', 'FORCE_CURVE', 'FORCE_BOID', 'FORCE_TURBULENCE', 'FORCE_DRAG', 'FORCE_FLUIDFLOW', 'RIGID_BODY', 'RIGID_BODY_CONSTRAINT', 'IMAGE_PLANE', 'IMAGE_BACKGROUND', 'IMAGE_REFERENCE', 'NODE_INSERT_ON', 'NODE_INSERT_OFF', 'NODE_TOP', 'NODE_SIDE', 'NODE_CORNER', 'ANCHOR_TOP', 'ANCHOR_BOTTOM', 'ANCHOR_LEFT', 'ANCHOR_RIGHT', 'ANCHOR_CENTER', 'SELECT_SET', 'SELECT_EXTEND', 'SELECT_SUBTRACT', 'SELECT_INTERSECT', 'SELECT_DIFFERENCE', 'ALIGN_LEFT', 'ALIGN_CENTER', 'ALIGN_RIGHT', 'ALIGN_JUSTIFY', 'ALIGN_FLUSH', 'ALIGN_TOP', 'ALIGN_MIDDLE', 'ALIGN_BOTTOM', 'BOLD', 'ITALIC', 'UNDERLINE', 'SMALL_CAPS', 'CON_ACTION', 'MOD_LENGTH', 'MOD_DASH', 'MOD_LINEART', 'HOLDOUT_OFF', 'HOLDOUT_ON', 'INDIRECT_ONLY_OFF', 'INDIRECT_ONLY_ON', 'CON_CAMERASOLVER', 'CON_FOLLOWTRACK', 'CON_OBJECTSOLVER', 'CON_LOCLIKE', 'CON_ROTLIKE', 'CON_SIZELIKE', 'CON_TRANSLIKE', 'CON_DISTLIMIT', 'CON_LOCLIMIT', 'CON_ROTLIMIT', 'CON_SIZELIMIT', 'CON_SAMEVOL', 'CON_TRANSFORM', 'CON_TRANSFORM_CACHE', 'CON_CLAMPTO', 'CON_KINEMATIC', 'CON_LOCKTRACK', 'CON_SPLINEIK', 'CON_STRETCHTO', 'CON_TRACKTO', 'CON_ARMATURE', 'CON_CHILDOF', 'CON_FLOOR', 'CON_FOLLOWPATH', 'CON_PIVOT', 'CON_SHRINKWRAP', 'MODIFIER_DATA', 'MOD_WAVE', 'MOD_BUILD', 'MOD_DECIM', 'MOD_MIRROR', 'MOD_SOFT', 'MOD_SUBSURF', 'HOOK', 'MOD_PHYSICS', 'MOD_PARTICLES', 'MOD_BOOLEAN', 'MOD_EDGESPLIT', 'MOD_ARRAY', 'MOD_UVPROJECT', 'MOD_DISPLACE', 'MOD_CURVE', 'MOD_LATTICE', 'MOD_TINT', 'MOD_ARMATURE', 'MOD_SHRINKWRAP', 'MOD_CAST', 'MOD_MESHDEFORM', 'MOD_BEVEL', 'MOD_SMOOTH', 'MOD_SIMPLEDEFORM', 'MOD_MASK', 'MOD_CLOTH', 'MOD_EXPLODE', 'MOD_FLUIDSIM', 'MOD_MULTIRES', 'MOD_FLUID', 'MOD_SOLIDIFY', 'MOD_SCREW', 'MOD_VERTEX_WEIGHT', 'MOD_DYNAMICPAINT', 'MOD_REMESH', 'MOD_OCEAN', 'MOD_WARP', 'MOD_SKIN', 'MOD_TRIANGULATE', 'MOD_WIREFRAME', 'MOD_DATA_TRANSFER', 'MOD_NORMALEDIT', 'MOD_PARTICLE_INSTANCE', 'MOD_HUE_SATURATION', 'MOD_NOISE', 'MOD_OFFSET', 'MOD_SIMPLIFY', 'MOD_THICKNESS', 'MOD_INSTANCE', 'MOD_TIME', 'MOD_OPACITY', 'REC', 'PLAY', 'FF', 'REW', 'PAUSE', 'PREV_KEYFRAME', 'NEXT_KEYFRAME', 'PLAY_SOUND', 'PLAY_REVERSE', 'PREVIEW_RANGE', 'ACTION_TWEAK', 'PMARKER_ACT', 'PMARKER_SEL', 'PMARKER', 'MARKER_HLT', 'MARKER', 'KEYFRAME_HLT', 'KEYFRAME', 'KEYINGSET', 'KEY_DEHLT', 'KEY_HLT', 'MUTE_IPO_OFF', 'MUTE_IPO_ON', 'DRIVER', 'SOLO_OFF', 'SOLO_ON', 'FRAME_PREV', 'FRAME_NEXT', 'NLA_PUSHDOWN', 'IPO_CONSTANT', 'IPO_LINEAR', 'IPO_BEZIER', 'IPO_SINE', 'IPO_QUAD', 'IPO_CUBIC', 'IPO_QUART', 'IPO_QUINT', 'IPO_EXPO', 'IPO_CIRC', 'IPO_BOUNCE', 'IPO_ELASTIC', 'IPO_BACK', 'IPO_EASE_IN', 'IPO_EASE_OUT', 'IPO_EASE_IN_OUT', 'NORMALIZE_FCURVES', 'VERTEXSEL', 'EDGESEL', 'FACESEL', 'CURSOR', 'PIVOT_BOUNDBOX', 'PIVOT_CURSOR', 'PIVOT_INDIVIDUAL', 'PIVOT_MEDIAN', 'PIVOT_ACTIVE', 'CENTER_ONLY', 'ROOTCURVE', 'SMOOTHCURVE', 'SPHERECURVE', 'INVERSESQUARECURVE', 'SHARPCURVE', 'LINCURVE', 'NOCURVE', 'RNDCURVE', 'PROP_OFF', 'PROP_ON', 'PROP_CON', 'PROP_PROJECTED', 'PARTICLE_POINT', 'PARTICLE_TIP', 'PARTICLE_PATH', 'SNAP_FACE_CENTER', 'SNAP_PERPENDICULAR', 'SNAP_MIDPOINT', 'SNAP_OFF', 'SNAP_ON', 'SNAP_NORMAL', 'SNAP_GRID', 'SNAP_VERTEX', 'SNAP_EDGE', 'SNAP_FACE', 'SNAP_VOLUME', 'SNAP_INCREMENT', 'STICKY_UVS_LOC', 'STICKY_UVS_DISABLE', 'STICKY_UVS_VERT', 'CLIPUV_DEHLT', 'CLIPUV_HLT', 'SNAP_PEEL_OBJECT', 'GRID', 'OBJECT_ORIGIN', 'ORIENTATION_GLOBAL', 'ORIENTATION_GIMBAL', 'ORIENTATION_LOCAL', 'ORIENTATION_NORMAL', 'ORIENTATION_VIEW', 'COPYDOWN', 'PASTEDOWN', 'PASTEFLIPUP', 'PASTEFLIPDOWN', 'VIS_SEL_11', 'VIS_SEL_10', 'VIS_SEL_01', 'VIS_SEL_00', 'AUTOMERGE_OFF', 'AUTOMERGE_ON', 'UV_VERTEXSEL', 'UV_EDGESEL', 'UV_FACESEL', 'UV_ISLANDSEL', 'UV_SYNC_SELECT', 'GP_CAPS_FLAT', 'GP_CAPS_ROUND', 'FIXED_SIZE', 'TRANSFORM_ORIGINS', 'GIZMO', 'ORIENTATION_CURSOR', 'NORMALS_VERTEX', 'NORMALS_FACE', 'NORMALS_VERTEX_FACE', 'SHADING_BBOX', 'SHADING_WIRE', 'SHADING_SOLID', 'SHADING_RENDERED', 'SHADING_TEXTURE', 'OVERLAY', 'XRAY', 'LOCKVIEW_OFF', 'LOCKVIEW_ON', 'AXIS_SIDE', 'AXIS_FRONT', 'AXIS_TOP', 'LAYER_USED', 'LAYER_ACTIVE', 'OUTLINER_OB_HAIR', 'OUTLINER_DATA_HAIR', 'HAIR_DATA', 'OUTLINER_OB_POINTCLOUD', 'OUTLINER_DATA_POINTCLOUD', 'POINTCLOUD_DATA', 'OUTLINER_OB_VOLUME', 'OUTLINER_DATA_VOLUME', 'VOLUME_DATA', 'CURRENT_FILE', 'HOME', 'DOCUMENTS', 'TEMP', 'SORTALPHA', 'SORTBYEXT', 'SORTTIME', 'SORTSIZE', 'SHORTDISPLAY', 'LONGDISPLAY', 'IMGDISPLAY', 'BOOKMARKS', 'FONTPREVIEW', 'FILTER', 'NEWFOLDER', 'FOLDER_REDIRECT', 'FILE_PARENT', 'FILE_REFRESH', 'FILE_FOLDER', 'FILE_BLANK', 'FILE_BLEND', 'FILE_IMAGE', 'FILE_MOVIE', 'FILE_SCRIPT', 'FILE_SOUND', 'FILE_FONT', 'FILE_TEXT', 'SORT_DESC', 'SORT_ASC', 'LINK_BLEND', 'APPEND_BLEND', 'IMPORT', 'EXPORT', 'LOOP_BACK', 'LOOP_FORWARDS', 'BACK', 'FORWARD', 'FILE_ARCHIVE', 'FILE_CACHE', 'FILE_VOLUME', 'FILE_3D', 'FILE_HIDDEN', 'FILE_BACKUP', 'DISK_DRIVE', 'MATPLANE', 'MATSPHERE', 'MATCUBE', 'MONKEY', 'HAIR', 'ALIASED', 'ANTIALIASED', 'MAT_SPHERE_SKY', 'MATSHADERBALL', 'MATCLOTH', 'MATFLUID', 'WORDWRAP_OFF', 'WORDWRAP_ON', 'SYNTAX_OFF', 'SYNTAX_ON', 'LINENUMBERS_OFF', 'LINENUMBERS_ON', 'SCRIPTPLUGINS', 'DISC', 'DESKTOP', 'EXTERNAL_DRIVE', 'NETWORK_DRIVE', 'SEQ_SEQUENCER', 'SEQ_PREVIEW', 'SEQ_LUMA_WAVEFORM', 'SEQ_CHROMA_SCOPE', 'SEQ_HISTOGRAM', 'SEQ_SPLITVIEW', 'SEQ_STRIP_META', 'SEQ_STRIP_DUPLICATE', 'IMAGE_RGB', 'IMAGE_RGB_ALPHA', 'IMAGE_ALPHA', 'IMAGE_ZDEPTH', 'HANDLE_AUTOCLAMPED', 'HANDLE_AUTO', 'HANDLE_ALIGNED', 'HANDLE_VECTOR', 'HANDLE_FREE', 'VIEW_PERSPECTIVE', 'VIEW_ORTHO', 'VIEW_CAMERA', 'VIEW_PAN', 'VIEW_ZOOM', 'BRUSH_BLOB', 'BRUSH_BLUR', 'BRUSH_CLAY', 'BRUSH_CLAY_STRIPS', 'BRUSH_CLONE', 'BRUSH_CREASE', 'BRUSH_FILL', 'BRUSH_FLATTEN', 'BRUSH_GRAB', 'BRUSH_INFLATE', 'BRUSH_LAYER', 'BRUSH_MASK', 'BRUSH_MIX', 'BRUSH_NUDGE', 'BRUSH_PINCH', 'BRUSH_SCRAPE', 'BRUSH_SCULPT_DRAW', 'BRUSH_SMEAR', 'BRUSH_SMOOTH', 'BRUSH_SNAKE_HOOK', 'BRUSH_SOFTEN', 'BRUSH_TEXDRAW', 'BRUSH_TEXFILL', 'BRUSH_TEXMASK', 'BRUSH_THUMB', 'BRUSH_ROTATE', 'GPBRUSH_SMOOTH', 'GPBRUSH_THICKNESS', 'GPBRUSH_STRENGTH', 'GPBRUSH_GRAB', 'GPBRUSH_PUSH', 'GPBRUSH_TWIST', 'GPBRUSH_PINCH', 'GPBRUSH_RANDOMIZE', 'GPBRUSH_CLONE', 'GPBRUSH_WEIGHT', 'GPBRUSH_PENCIL', 'GPBRUSH_PEN', 'GPBRUSH_INK', 'GPBRUSH_INKNOISE', 'GPBRUSH_BLOCK', 'GPBRUSH_MARKER', 'GPBRUSH_FILL', 'GPBRUSH_AIRBRUSH', 'GPBRUSH_CHISEL', 'GPBRUSH_ERASE_SOFT', 'GPBRUSH_ERASE_HARD', 'GPBRUSH_ERASE_STROKE', 'KEYTYPE_KEYFRAME_VEC', 'KEYTYPE_BREAKDOWN_VEC', 'KEYTYPE_EXTREME_VEC', 'KEYTYPE_JITTER_VEC', 'KEYTYPE_MOVING_HOLD_VEC', 'HANDLETYPE_FREE_VEC', 'HANDLETYPE_ALIGNED_VEC', 'HANDLETYPE_VECTOR_VEC', 'HANDLETYPE_AUTO_VEC', 'HANDLETYPE_AUTO_CLAMP_VEC', 'COLORSET_01_VEC', 'COLORSET_02_VEC', 'COLORSET_03_VEC', 'COLORSET_04_VEC', 'COLORSET_05_VEC', 'COLORSET_06_VEC', 'COLORSET_07_VEC', 'COLORSET_08_VEC', 'COLORSET_09_VEC', 'COLORSET_10_VEC', 'COLORSET_11_VEC', 'COLORSET_12_VEC', 'COLORSET_13_VEC', 'COLORSET_14_VEC', 'COLORSET_15_VEC', 'COLORSET_16_VEC', 'COLORSET_17_VEC', 'COLORSET_18_VEC', 'COLORSET_19_VEC', 'COLORSET_20_VEC', 'COLLECTION_COLOR_01', 'COLLECTION_COLOR_02', 'COLLECTION_COLOR_03', 'COLLECTION_COLOR_04', 'COLLECTION_COLOR_05', 'COLLECTION_COLOR_06', 'COLLECTION_COLOR_07', 'COLLECTION_COLOR_08', 'SEQUENCE_COLOR_01', 'SEQUENCE_COLOR_02', 'SEQUENCE_COLOR_03', 'SEQUENCE_COLOR_04', 'SEQUENCE_COLOR_05', 'SEQUENCE_COLOR_06', 'SEQUENCE_COLOR_07', 'SEQUENCE_COLOR_08', 'SEQUENCE_COLOR_09', 'EVENT_A', 'EVENT_B', 'EVENT_C', 'EVENT_D', 'EVENT_E', 'EVENT_F', 'EVENT_G', 'EVENT_H', 'EVENT_I', 'EVENT_J', 'EVENT_K', 'EVENT_L', 'EVENT_M', 'EVENT_N', 'EVENT_O', 'EVENT_P', 'EVENT_Q', 'EVENT_R', 'EVENT_S', 'EVENT_T', 'EVENT_U', 'EVENT_V', 'EVENT_W', 'EVENT_X', 'EVENT_Y', 'EVENT_Z', 'EVENT_SHIFT', 'EVENT_CTRL', 'EVENT_ALT', 'EVENT_OS', 'EVENT_F1', 'EVENT_F2', 'EVENT_F3', 'EVENT_F4', 'EVENT_F5', 'EVENT_F6', 'EVENT_F7', 'EVENT_F8', 'EVENT_F9', 'EVENT_F10', 'EVENT_F11', 'EVENT_F12', 'EVENT_ESC', 'EVENT_TAB', 'EVENT_PAGEUP', 'EVENT_PAGEDOWN', 'EVENT_RETURN', 'EVENT_SPACEKEY')

In [95]:
sorted(l)

['ACTION',
 'ACTION_TWEAK',
 'ADD',
 'ALIASED',
 'ALIGN_BOTTOM',
 'ALIGN_CENTER',
 'ALIGN_FLUSH',
 'ALIGN_JUSTIFY',
 'ALIGN_LEFT',
 'ALIGN_MIDDLE',
 'ALIGN_RIGHT',
 'ALIGN_TOP',
 'ANCHOR_BOTTOM',
 'ANCHOR_CENTER',
 'ANCHOR_LEFT',
 'ANCHOR_RIGHT',
 'ANCHOR_TOP',
 'ANIM',
 'ANIM_DATA',
 'ANTIALIASED',
 'APPEND_BLEND',
 'ARMATURE_DATA',
 'ARROW_LEFTRIGHT',
 'ASSET_MANAGER',
 'AUTO',
 'AUTOMERGE_OFF',
 'AUTOMERGE_ON',
 'AXIS_FRONT',
 'AXIS_SIDE',
 'AXIS_TOP',
 'BACK',
 'BLANK1',
 'BLENDER',
 'BOIDS',
 'BOLD',
 'BONE_DATA',
 'BOOKMARKS',
 'BORDERMOVE',
 'BRUSHES_ALL',
 'BRUSH_BLOB',
 'BRUSH_BLUR',
 'BRUSH_CLAY',
 'BRUSH_CLAY_STRIPS',
 'BRUSH_CLONE',
 'BRUSH_CREASE',
 'BRUSH_DATA',
 'BRUSH_FILL',
 'BRUSH_FLATTEN',
 'BRUSH_GRAB',
 'BRUSH_INFLATE',
 'BRUSH_LAYER',
 'BRUSH_MASK',
 'BRUSH_MIX',
 'BRUSH_NUDGE',
 'BRUSH_PINCH',
 'BRUSH_ROTATE',
 'BRUSH_SCRAPE',
 'BRUSH_SCULPT_DRAW',
 'BRUSH_SMEAR',
 'BRUSH_SMOOTH',
 'BRUSH_SNAKE_HOOK',
 'BRUSH_SOFTEN',
 'BRUSH_TEXDRAW',
 'BRUSH_TEXFILL',
 'BRUSH_T